## Отчет по лабораторной работе №3

Параметры ноутбука исследовались с помощью программы CPU-Z.

Скрины ниже показывают характеристики CPU и его кэша. Параметры остальных частей системы можно посмотреть в папке [cpu-z-screens](cpu-z-screens).

![Рисунок 1](cpu-z-screens/Specs_02_CPU.png)
![Рисунок 2](cpu-z-screens/Specs_01_Caches.png)

Код для анализа количества промахов находится в файлах [Cache.h](Cache.h) и [main_analyzed.cpp](main_analyzed.cpp). Компиляция производится с помощью скрипта [compile.sh](compile.sh). Получаем программу cache_analyzed, которая на вход получает следующие аргументы:
1. размер матрицы
+  размер кэша в байтах
+  ассоциативность кэша
+  размер кэш линии в байтах
+  стратегия вытеснения из кэша (0 - LRU, 1 - вытеснение произовльной линии)

Тестирование производилось с помощью скрипта [run_tests.sh](run_tests.sh), получены следующие файлы с результатами:
* cache\_analyzer\_lru\_*.log - для результатов работы написанного анализатора кэша со стратегией вытеснения LRU
* cache\_analyzer\_random\_*.log - для результатов работы написанного анализатора кэша со стратегией вытеснения произвольной линии
* valgrind_*.log - для результатов работы valgrind


Исследования проводились для двух вариантов:
1. использовались характеристики компьютера (размер кэш-линии - 6291456 = 6 Mb, ассоциативность - 12, кэш-линия - 64 байта)
+  использовались кастомные характеристики (размер кэш-линии - 3145728 = 3 Mb, ассоциативность - 12, кэш-линия - 64 байта)

Первый вариант соответствует результатам с постфиксом 6mb, второй - 3mb.

Представим результаты на графиках.